In [1]:
import nibabel as nib
#import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.cluster import KMeans
from os import listdir
from os.path import join, exists
from scipy.io import savemat,loadmat
from skimage.measure import label   


## Load stuff

In [2]:
directory = 'OsteoarthritisInitiative/my_predictions/nii_files'
directory_org = 'OsteoarthritisInitiative/NIFTY'


# Make empty list
b = os.listdir(directory)
b = [filename[:-12] for filename in b]

df_mri = pd.read_csv("mri_data_ent", index_col ="Unnamed: 0")

for i in set(df_mri.index): 
    if i[-2:]=="_s":
        df_mri = df_mri.drop([i])
        print("Dropped", i )

        
iindex = []
for i in b:  
    if i in df_mri.index:
        continue
        
    if i[-7:] == '_PRED_s':
        continue
    
    iindex.append(i)

if len(iindex)!=0:
    empty = pd.DataFrame(np.zeros((len(iindex),23)), columns = df_mri.columns, index = iindex)
    df_mri = df_mri.append(empty)

# help functions

In [3]:
# Helper function for visualizing the segmentation masks. 
def showMask(mask, ylabel=None, showCoM=None):
    sz = mask.shape
    if showCoM is not None:
        [xs, ys, zs] = np.nonzero(mask)
        x = np.round(np.mean(xs)).astype('int')
        y = np.round(np.mean(ys)).astype('int')
        z = np.round(np.mean(zs)).astype('int')
        d = 10
        cop = mask.copy()
        mask = cop
        mask[(x-d):(x+d), (y-d):(y+d), (z-d):(z+d)] = 1
    if True: # Mask is collapsed along the three scan axes
        fig, (ax0,ax1,ax2) = plt.subplots(1,3)
        ax0.imshow(np.sum(mask,axis=0))
        ax1.imshow(np.sum(mask,axis=1))
        ax2.imshow(np.sum(mask,axis=2))   
    else: # Also the center slice is shown for each axis
        fig, (ax0,ax1,ax2,ax3,ax4,ax5) = plt.subplots(1,6)
        ax0.imshow(mask[sz[0]//2,:,:])
        ax1.imshow(np.sum(mask,axis=0))
        ax2.imshow(mask[:,sz[1]//2,:])
        ax3.imshow(np.sum(mask,axis=1))
        ax4.imshow(mask[:,:,sz[2]//2])
        ax5.imshow(np.sum(mask,axis=2))
    if ylabel is not None:
        ax0.set_ylabel(ylabel)

In [4]:
def erosion(mask):
    ero = np.zeros(mask.shape)

    idx = np.argwhere(mask==1)
    #dialated[idx]=2
    for n in idx:
        i = n[0]
        j = n[1]
        k = n[2]
        
        if i+1 > mask.shape[0] or i==0:
            continue
        if j+1 > mask.shape[1] or j ==0:
            continue
        if k+1 > mask.shape[2] or k ==0:
            continue

        if mask[i+1,j,k]==1 and mask[i,j+1,k]==1 and mask[i,j,k+1]==1 and mask[i-1,j,k]==1 and mask[i,j-1,k]==1 and mask[i,j,k-1]==1:
            ero[i,j,k] = 1
                    
    return ero

#showMask(erosion(mask_fem))

In [5]:
def getLargestCC(segmentation):
    labels = label(segmentation)
    assert( labels.max() != 0 ) # assume at least 1 CC
    largestCC = labels == np.argmax(np.bincount(labels.flat)[1:])+1
    return largestCC

In [6]:
from skimage import measure

def get_amount_erosions(mask):
    img = getLargestCC(mask)
    n_comp = np.max(measure.label(img , background=0))
    n_erosion=0

    while n_comp == 1: 
        img = erosion(img)
        n_comp = np.max(measure.label(img , background=0))
        n_erosion += 1

    return n_erosion


In [7]:
def get_thinkness_along_axis(mask):
    weight = np.zeros(mask.shape)
    
    along_x = np.sum(mask, axis=(0))
    n_x = len(np.where(along_x!=0.)[0])
    
    along_y = np.sum(mask, axis=(1))
    n_y = len(np.where(along_y!=0.)[0])

    along_z = np.sum(mask, axis=(2))
    n_z = len(np.where(along_z!=0.)[0])

    return np.sum(along_x)/n_x, np.sum(along_y)/n_y, np.sum(along_z)/n_z

# Extract thicknesses

In [8]:
#df_mri[["n erosions Tibial", "n erosions Femur", "n erosions Meniscus","Thick Tibial Z","Thick Tibial Y","Thick Tibial X","Thick Meniscus X", "Thick Meniscus Y","Thick Meniscus Z","Thick Femur X","Thick Femur Y","Thick Femur Z"]]=0

In [12]:
# add amount of medial/lateral tibial
a=0
for idx in df_mri.index:
        a += 1
        

        if df_mri["Thick Meniscus Z"].loc[idx] != 0:
            continue
        
        print(idx)    
        path = directory+"/"+idx+"_PRED.nii.gz"
        seg = nib.load(path).get_fdata()
            
        mask_fem = np.where(seg==1,1,0)
        mask_tib = np.where(seg==2,1,0)
        mask_min = np.where(seg==4,1,0)

        x,y,z = get_thinkness_along_axis(mask_fem) 
        n = get_amount_erosions(mask_fem)
        df_mri.loc[idx, ("n erosions Femur")] = n
        df_mri.loc[idx, ("Thick Femur X")] = x
        df_mri.loc[idx, ("Thick Femur Y")] = y
        df_mri.loc[idx, ("Thick Femur Z")] = z

        x,y,z = get_thinkness_along_axis(mask_tib) 
        n = get_amount_erosions(mask_tib)
        df_mri.loc[idx, ("n erosions Tibial")] = n
        df_mri.loc[idx, ("Thick Tibial X")] = x
        df_mri.loc[idx, ("Thick Tibial Y")] = y
        df_mri.loc[idx, ("Thick Tibial Z")] = z
        
        x,y,z = get_thinkness_along_axis(mask_min) 
        n = get_amount_erosions(mask_min)
        df_mri.loc[idx, ("n erosions Meniscus")] = n        
        df_mri.loc[idx, ("Thick Meniscus X")] = x
        df_mri.loc[idx, ("Thick Meniscus Y")] = y
        df_mri.loc[idx, ("Thick Meniscus Z")] = z
      
        if a/5 == a//5:
            print(a)
            df_mri.to_csv("mri_data_ent")
            
df_mri.to_csv("mri_data_ent")

In [13]:
#8310

In [14]:
df_mri

,Cartilage: 1,Cartilage: 2,Cartilage: 3,Cartilage: 4,Medial Tibial,Lateral Tibial,Medial Femur,Lateral Femur,Medial Miniscus,Lateral Miniscus,...,n erosions Meniscus,Thick Tibial Z,Thick Tibial Y,Thick Tibial X,Thick Meniscus X,Thick Meniscus Y,Thick Meniscus Z,Thick Femur X,Thick Femur Y,Thick Femur Z
9000798-Left-V00,124138.0,61541.0,3014.0,58123.0,14070.0,47471.0,17503.0,49270.0,28974.0,29149.0,...,6,13.926454,14.638677,7.157595,9.463204,17.391682,16.140794,10.639184,10.208717,16.111356
9000798-Left-V01,99519.0,69283.0,44.0,52372.0,23863.0,45420.0,11896.0,41788.0,22278.0,30094.0,...,6,14.653765,13.801394,8.515610,9.444905,16.615482,17.850034,9.631182,9.328740,14.068278
9000798-Left-V03,103289.0,69378.0,115.0,52494.0,23555.0,45823.0,11688.0,44472.0,22698.0,29796.0,...,2,15.735541,12.989702,8.478309,9.290973,16.787336,17.958946,10.010564,9.389909,14.199753
9000798-Left-V05,93251.0,52852.0,1278.0,48753.0,41173.0,11679.0,26751.0,33046.0,20199.0,28554.0,...,4,13.692228,14.448332,7.327326,9.210845,15.691342,18.453066,9.720734,9.057012,14.466491
9000798-Left-V06,88185.0,62089.0,131.0,46197.0,25434.0,36655.0,7550.0,39146.0,17625.0,28572.0,...,1,14.127190,11.517158,8.684991,9.171531,16.029493,14.935984,9.388374,8.632048,12.123316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9968721-Right-V03,59886.0,32010.0,0.0,25386.0,14751.0,17259.0,18377.0,17506.0,10581.0,14805.0,...,3,12.996346,26.520298,5.300546,6.664741,14.025414,10.211585,7.023927,9.726490,11.746960
9973199-Right-V00,41315.0,30302.0,1505.0,35212.0,14202.0,16100.0,9290.0,18025.0,20796.0,14416.0,...,4,10.057086,22.800602,4.884268,7.643152,17.819838,10.938801,4.605909,7.173989,8.322925
9985803-Left-V03,85246.0,37407.0,43.0,28855.0,17098.0,20309.0,23559.0,26285.0,14167.0,14688.0,...,1,12.427575,30.536327,5.050905,6.794208,18.484946,12.432141,6.608217,8.617671,10.622555
9985803-Right-V03,76106.0,37867.0,2253.0,32270.0,19468.0,18399.0,17301.0,34718.0,17446.0,14824.0,...,4,12.266602,29.887135,5.010851,6.929354,16.347518,11.070326,6.929436,7.829835,10.292940


In [11]:
df_mri= df_mri.drop(["9023407-Left-V01"])
df_mri= df_mri.drop(["9025994-Left-V06"])
df_mri= df_mri.drop(["9061827-Right-V05"])